## 1.1 Function definition:

In [1]:
import pandas as pd
df_global_debug = pd.DataFrame([])
DEBUG_ME = False

def ScrapeSingleStock(stock_name,stock_url,report_folder):
    import pandas as pd 
    from bs4 import BeautifulSoup
    import requests as rq
    import re
    import datetime
    import os
    import time

    # local variables
    stock_sector                = ""
    stock_industry              = ""
    stock_url_type              = ""
    df_basic_info               = pd.DataFrame([]) # Basic Info
    df_top_ratios               = pd.DataFrame([]) # Top Ratios
    df_quaterly_results         = pd.DataFrame([]) # Quarterly Results
    df_profit_n_loss            = pd.DataFrame([]) # Profit & Loss
    df_compounded_sales_growth  = pd.DataFrame([]) # Compounded Sales Growth
    df_compounded_profit_growth = pd.DataFrame([]) # Compounded Profit Growth
    df_stock_price_cagr         = pd.DataFrame([]) # Stock Price CAGR
    df_return_on_equity         = pd.DataFrame([]) # Return on Equity
    df_balance_sheet            = pd.DataFrame([]) # Balance Sheet
    df_cash_flows               = pd.DataFrame([]) # Cash Flows
    df_ratios                   = pd.DataFrame([]) # Ratios
    df_shareholding_pattern     = pd.DataFrame([]) # Shareholding Pattern
    writer                      = None
    report_name                 = ""
    stock_reportpath            = ""
    

    if(DEBUG_ME): print("Info: Starting ScrapeStock()")
    if(DEBUG_ME): print("Info: Stock: ", stock_name)
    if(DEBUG_ME): print("Info: URL: ", stock_url)
    if(DEBUG_ME): print("Info: Report Folder: ", report_folder)

    if not os.path.exists(report_folder):
        os.makedirs(report_folder)
        if(DEBUG_ME): print("Info: Folder created: ",report_folder)


    if 'consolidated' in stock_url:
        stock_url_type = "consolidated"
        if(DEBUG_ME): print("Info: stock_url_type is : ",stock_url_type)
    elif 'consolidated' not in stock_url:
        stock_url_type = "standalone"
        if(DEBUG_ME): print("Info: stock_url_type is : ",stock_url_type)
    else:
        stock_url_type = ""
        if(DEBUG_ME): print("Info: stock_url_type is : ",stock_url_type)


    # Read the page: 
    if(DEBUG_ME): print("Info: Fetching URL...")
    response = rq.get(stock_url)
    if(DEBUG_ME): print("Info: URL Fetch completed")
    if(DEBUG_ME): print("Info: Response status code: ",response.status_code)
    time.sleep(1) # Seconds

    tables = pd.read_html(response.content)
    if(DEBUG_ME): print("Info: Parsing Tables using Pandas. Number of Tables found: ",str(len(tables)))

    if(DEBUG_ME): print("Info: beginning data parsing")

    soup = BeautifulSoup(response.content, "html.parser")
    if(DEBUG_ME): print("Info: BSoup Object loaded")

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Parsing: Sector and industry: Start")
    temp0 = soup.findAll('p', attrs={'class':'sub'})[1].text
    temp0 = ' '.join(temp0.split())

    if m := re.match(r'Sector:\s*([A-Za-z ]*)Industry:\s*([A-Za-z- ]*)$', temp0):
        stock_sector = m.group(1).strip()
        stock_industry = m.group(2).strip()
        if(DEBUG_ME): print("Info: Parsing: Sector and industry: Found Sector:", stock_sector)
        if(DEBUG_ME): print("Info: Parsing: Sector and industry: Found Industry:", stock_industry)
    else: 
        stock_sector = 'Null'
        stock_industry = 'Null'
        if(DEBUG_ME): print("Error: Parsing: Failed to find Sector and Industry!")
    if(DEBUG_ME): print("Info: Parsing: Sector and industry: Finished")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Parsing: Start - Basic Info")

    temp_1 = [{
    "Stockname":stock_name,
    "Stock Report Type": stock_url_type,
    "Stock Url": stock_url,
    "Sector": stock_sector,
    "Industry": stock_industry
    }]

    df_basic_info = pd.DataFrame(temp_1)

    if(DEBUG_ME): print("Info: Parsing: Finished - Basic Info")
    ## ---------------------------------------------------------

    


    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Parsing: Start - Top Ratios")

    ul = soup.find("ul#top-ratios")  # Selector of top ratios
    lines = []
    for ul in soup.findAll('ul', id='top-ratios'):
        for li in ul.findAll('li'):
            li_parsed_text = li.text
            li_parsed_text = re.sub('[\s ]+', ' ', li_parsed_text)
            li_parsed_text = li_parsed_text.strip()
            #print(li_parsed_text)
            lines.append(li_parsed_text)

    if(DEBUG_ME): print("Info: Parsing: Top Ratios: Cleaning up the To Ratios data line by line")
     # Line - 0
    if m := re.match(r"Market Cap ₹ ([0-9,.-]+) Cr.", lines[0]):
        line_0 = m.group(1).replace(',','')
        if(DEBUG_ME): print("Info: Parsing: Top Ratios: Found Market Cap: ", line_0)
    else: 
        line_0 = "NaN"
        if(DEBUG_ME): print("Error: Parsing: Top Ratios: Not Found Market Cap")

    # Line - 1
    if m := re.match(r"Current Price ₹ ([0-9,.-]+)", lines[1]):
        line_1 = m.group(1).replace(',','')
        if(DEBUG_ME): print("Info: Parsing: Top Ratios: Found Current Price: ", line_1)
    else: 
        line_1 = "NaN"

    # Line - 2a
    if m := re.match(r"High \/ Low ₹ ([0-9,..-]+) \/ ([0-9,.]+)", lines[2]):
        line_2a = m.group(1).replace(',','')
        if(DEBUG_ME): print("Info: Parsing: Top Ratios: Found High: ", line_2a)
    else: 
        line_2a = "NaN"
        if(DEBUG_ME): print("Error: Parsing: Top Ratios: Not Found High")

    # Line - 2b
    if m := re.match(r"High \/ Low ₹ ([0-9,.-]+) \/ ([0-9,.]+)", lines[2]):
        line_2b = m.group(2).replace(',','')
        if(DEBUG_ME): print("Info: Parsing: Top Ratios: Found Low: ", line_2b)
    else: 
        line_2b = "NaN"
        if(DEBUG_ME): print("Error: Parsing: Top Ratios: Not Found Low")

    # Line - 3
    if m := re.match(r"Stock P\/E ([0-9,.-]+)", lines[3]):
        line_3 = m.group(1).replace(',','')
        if(DEBUG_ME): print("Info: Parsing: Top Ratios: Found Low: ", line_3)
    else: 
        line_3 = "NaN"
        if(DEBUG_ME): print("Error: Parsing: Top Ratios: Not Found __")

    # Line - 4
    if m := re.match(r"Book Value ₹ ([0-9,.-]+)", lines[4]):
        line_4 = m.group(1).replace(',','')
        if(DEBUG_ME): print("Info: Parsing: Top Ratios: Found Book Value: ", line_4)
    else: 
        line_4 = "NaN"
        if(DEBUG_ME): print("Error: Parsing: Top Ratios: Not Found Book Value")

    # Line - 5
    if m := re.match(r"Dividend Yield ([0-9,.-]+) %", lines[5]):
        line_5 = m.group(1).replace(',','')
        if(DEBUG_ME): print("Info: Parsing: Top Ratios: Found Dividend Yield: ", line_5)
    else: 
        line_5 = "NaN"
        if(DEBUG_ME): print("Error: Parsing: Top Ratios: Not Found Dividend Yield")

    # Line - 6
    if m := re.match(r"ROCE ([0-9,.-]+) %", lines[6]):
        line_6 = m.group(1).replace(',','')
        if(DEBUG_ME): print("Info: Parsing: Top Ratios: Found ROCE: ", line_6)
    else: 
        line_6 = "NaN"
        if(DEBUG_ME): print("Error: Parsing: Top Ratios: Not Found ROCE")

    # Line - 7
    if m := re.match(r"ROE ([0-9,.-]+) %", lines[7]):
        line_7 = m.group(1).replace(',','')
        if(DEBUG_ME): print("Info: Parsing: Top Ratios: Found ROE: ", line_7)
    else: 
        line_7 = "NaN"
        if(DEBUG_ME): print("Error: Parsing: Top Ratios: Not Found ROE")

    # Line - 8
    if m := re.match(r"Face Value ₹ ([0-9,.-]+)", lines[8]):
        line_8 = m.group(1).replace(',','')
        if(DEBUG_ME): print("Info: Parsing: Top Ratios: Found Face Value: ", line_8)
    else: 
        line_8 = "NaN"
        if(DEBUG_ME): print("Error: Parsing: Top Ratios: Not Found Face Value")

    if(DEBUG_ME): print("Info: Parsing: Top Ratios: Preparing Dataframe")
    temp_2 = [{
    "Stockname":stock_name,
    "Market Cap in Cores Rupees":line_0,
    "Current Price in Rupees":line_1,
    "High in Rupees":line_2a,
    "Low in Rupees":line_2b,
    "Stock PE":line_3,
    "Book Value in Rupees":line_4,
    "Dividend Yield %": line_5,
    "ROCE %":line_6,
    "ROE %":line_7,
    "Face Value in Rupees":line_8,
    }]

    df_top_ratios = pd.DataFrame(temp_2)
    if(DEBUG_ME): print("Info: Parsing: Top Ratios: Done Dataframe for Top Ratios")

    if(DEBUG_ME): print("Info: Parsing: Finished - Top Ratios")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Parsing: Start - All Pandas Tables")
    
    df_quaterly_results         = tables[0] # Quarterly Results
    df_profit_n_loss            = tables[1] # Profit & Loss
    df_compounded_sales_growth  = tables[2] # Compounded Sales Growth
    df_compounded_profit_growth = tables[3] # Compounded Profit Growth
    df_stock_price_cagr         = tables[4] # Stock Price CAGR
    df_return_on_equity         = tables[5] # Return on Equity
    df_balance_sheet            = tables[6] # Balance Sheet
    df_cash_flows               = tables[7] # Cash Flows
    df_ratios                   = tables[8] # Ratios
    df_shareholding_pattern     = tables[9] # Shareholding Pattern

    if(DEBUG_ME): print("Info: Parsing: Finished - All Pandas Tables")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Cleaning: Start - All Pandas Tables")
    
    # Cleanup table: Quarterly Results
    df_quaterly_results.rename(columns={'Unnamed: 0':'Quarterly Results'}, inplace=True)
    df_quaterly_results.replace(u"\u00A0\+", "", regex=True,inplace=True)
    if(DEBUG_ME): print("Info: Cleaning: Done - Quarterly Results")
    
    # Cleanup table: Profit & Loss
    df_profit_n_loss.rename(columns={'Unnamed: 0':'Profit and Loss'}, inplace=True)
    df_profit_n_loss.replace(u"\u00A0\+", "", regex=True,inplace=True)
    if(DEBUG_ME): print("Info: Cleaning: Done - Profit & Loss")

    # Cleanup table: Compounded Sales Growth
    df_compounded_sales_growth.replace(":", "", regex=True,inplace=True) 
    
    # Cleanup table: Compounded Profit Growth
    df_compounded_profit_growth.replace(":", "", regex=True,inplace=True) 
    
    # Cleanup table: Stock Price CAGR
    df_stock_price_cagr.replace(":", "", regex=True,inplace=True) 
    
    # Cleanup table: Return on Equity
    df_return_on_equity.replace(":", "", regex=True,inplace=True) 
    
    # Cleanup table: Balance Sheet
    df_balance_sheet.rename(columns={'Unnamed: 0':'Balance Sheet'}, inplace=True)
    df_balance_sheet.replace(u"\u00A0\+", "", regex=True,inplace=True) 
    
    # Cleanup table: Cash Flows
    df_cash_flows.rename(columns={'Unnamed: 0':'Cash Flows'}, inplace=True)
    df_cash_flows.replace(u"\u00A0\+", "", regex=True,inplace=True) 
    
    # Cleanup table: Ratios
    df_ratios.rename(columns={'Unnamed: 0':'Ratios'}, inplace=True)
    
    # Cleanup table: Shareholding Pattern
    df_shareholding_pattern.rename(columns={'Unnamed: 0':'Shareholding Pattern'}, inplace=True)
    df_shareholding_pattern.replace(u"\u00A0\+", "", regex=True,inplace=True)

    if(DEBUG_ME): print("Info: Cleaning: Finished - All Pandas Tables")
    ## ---------------------------------------------------------

    ## =================================================================================
    ## Dataframe contains space, non-numeric and percent sign so those should be removed
    ## =================================================================================
    
    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_top_ratios")
    # First col is texual, rest are numeric
    #df_top_ratios[df_top_ratios.columns[1:]] = df_top_ratios[df_top_ratios.columns[1:]].apply(pd.to_numeric)

    for col in df_top_ratios.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        df_top_ratios[col] = df_top_ratios[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_top_ratios[col] = df_top_ratios[col].apply(pd.to_numeric)

    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",df_top_ratios.info())
    if(DEBUG_ME): print("Info: Data Transform: Done - df_top_ratios")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_quaterly_results")
    # First col is texual, rest are numeric
    for col in df_quaterly_results.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        df_quaterly_results[col] = df_quaterly_results[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_global_debug[col] = df_quaterly_results[col]
        df_quaterly_results[col] = df_quaterly_results[col].apply(pd.to_numeric)

    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",temp_3)
    if(DEBUG_ME): print("Info: Data Transform: Done - df_quaterly_results")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_profit_n_loss")
    # First col is texual, rest are numeric
    for col in df_profit_n_loss.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        df_profit_n_loss[col] = df_profit_n_loss[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_profit_n_loss[col] = df_profit_n_loss[col].apply(pd.to_numeric)
    
    temp_3 = df_profit_n_loss.info()
    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",temp_3)
    if(DEBUG_ME): print("Info: Data Transform: Done - df_profit_n_loss")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_compounded_sales_growth")
    # First col is texual, rest are numeric
    for col in df_compounded_sales_growth.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        df_compounded_sales_growth[col] = df_compounded_sales_growth[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_compounded_sales_growth[col] = df_compounded_sales_growth[col].apply(pd.to_numeric)
    
    temp_3 = df_compounded_sales_growth.info()
    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",temp_3)
    if(DEBUG_ME): print("Info: Data Transform: Done - df_compounded_sales_growth")
    ## ---------------------------------------------------------

    
    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_compounded_profit_growth")
    # First col is texual, rest are numeric
    for col in df_compounded_profit_growth.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        df_compounded_profit_growth[col] = df_compounded_profit_growth[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_compounded_profit_growth[col] = df_compounded_profit_growth[col].apply(pd.to_numeric)
    
    temp_3 = df_compounded_profit_growth.info()
    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",temp_3)
    if(DEBUG_ME): print("Info: Data Transform: Done - df_compounded_profit_growth")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_stock_price_cagr")
    # First col is texual, rest are numeric
    for col in df_stock_price_cagr.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        df_stock_price_cagr[col] = df_stock_price_cagr[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_stock_price_cagr[col] = df_stock_price_cagr[col].apply(pd.to_numeric)
    
    temp_3 = df_stock_price_cagr.info()
    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",temp_3)
    if(DEBUG_ME): print("Info: Data Transform: Done - df_stock_price_cagr")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_return_on_equity")
    # First col is texual, rest are numeric
    for col in df_return_on_equity.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        df_return_on_equity[col] = df_return_on_equity[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_return_on_equity[col] = df_return_on_equity[col].apply(pd.to_numeric)
    
    temp_3 = df_return_on_equity.info()
    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",temp_3)
    if(DEBUG_ME): print("Info: Data Transform: Done - df_return_on_equity")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_balance_sheet")
    # First col is texual, rest are numeric
    for col in df_balance_sheet.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        ## CAUTION: Converting value to string!
        df_balance_sheet[col] = df_balance_sheet[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_balance_sheet[col] = df_balance_sheet[col].apply(pd.to_numeric)
    
    temp_3 = df_balance_sheet.info()
    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",temp_3)
    if(DEBUG_ME): print("Info: Data Transform: Done - df_balance_sheet")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_cash_flows")
    # First col is texual, rest are numeric
    for col in df_cash_flows.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        df_cash_flows[col] = df_cash_flows[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_cash_flows[col] = df_cash_flows[col].apply(pd.to_numeric)
    
    temp_3 = df_cash_flows.info()
    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",temp_3)
    if(DEBUG_ME): print("Info: Data Transform: Done - df_cash_flows")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_ratios")
    # First col is texual, rest are numeric
    for col in df_ratios.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        df_ratios[col] = df_ratios[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_ratios[col] = df_ratios[col].apply(pd.to_numeric)
    
    temp_3 = df_ratios.info()
    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",temp_3)
    if(DEBUG_ME): print("Info: Data Transform: Done - df_ratios")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Data Transform: Start - df_shareholding_pattern")
    # First col is texual, rest are numeric
    for col in df_shareholding_pattern.columns[1:]:
        # Remove comma, percent sign and Empty value with NaN
        df_shareholding_pattern[col] = df_shareholding_pattern[col].astype(str).str.replace(',','').str.rstrip('%').replace('nan','NaN').replace('','NaN').replace("NaN", pd.NA)
        df_shareholding_pattern[col] = df_shareholding_pattern[col].apply(pd.to_numeric)
    
    temp_3 = df_shareholding_pattern.info()
    if(DEBUG_ME): print("Info: Data Transform: Outcome: \n",temp_3)
    if(DEBUG_ME): print("Info: Data Transform: Done - df_shareholding_pattern")
    ## ---------------------------------------------------------

    ## ---------------------------------------------------------
    if(DEBUG_ME): print("Info: Saving Report...")

    sheet_names = ["Basic Info", "Top Ratios","Quarterly Results", "Profit & Loss", "Compounded Sales Growth", "Compounded Profit Growth", 
                   "Stock Price CAGR", "Return on Equity", "Balance Sheet", "Cash Flows", "Ratios", "Shareholding Pattern"]
    dataframes  = [df_basic_info, df_top_ratios, df_quaterly_results, df_profit_n_loss , df_compounded_sales_growth, df_compounded_profit_growth, 
                   df_stock_price_cagr, df_return_on_equity, df_balance_sheet, df_cash_flows, df_ratios, df_shareholding_pattern]

    if 'consolidated' in stock_url:
        report_name = stock_name + "-consolidated-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".xlsx"
        if(DEBUG_ME): print("Info: report name will be: ",report_name)
    elif 'consolidated' not in stock_url:
        report_name = stock_name + "-standalone-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".xlsx"
        if(DEBUG_ME): print("Info: report name suffix will be: ",report_name)
    else:
        report_name = stock_name + "-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".xlsx"
        if(DEBUG_ME): print("Info: report name suffix will be: ",report_name)
    
    
    stock_reportpath = report_folder + "/" + report_name
    
    writer = pd.ExcelWriter(stock_reportpath, engine='xlsxwriter')
    
    for i, frame in enumerate(dataframes):
        frame.to_excel(writer, sheet_name = sheet_names[i], index=False)
    writer.close()
    writer.handles = None

    if(DEBUG_ME): print("Info: Report saved successfully: ", stock_reportpath)
    ## ---------------------------------------------------------

    
    return stock_reportpath

## 1.2 Function Call: 

In [2]:
stock_name = "TCS"
stock_url = "https://www.screener.in/company/TCS/consolidated/"
report_folder = "../Output/RawReports-03"

ScrapeSingleStock(stock_name,stock_url,report_folder)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Stockname                   1 non-null      object 
 1   Market Cap in Cores Rupees  1 non-null      int64  
 2   Current Price in Rupees     1 non-null      int64  
 3   High in Rupees              1 non-null      int64  
 4   Low in Rupees               1 non-null      int64  
 5   Stock PE                    1 non-null      float64
 6   Book Value in Rupees        1 non-null      int64  
 7   Dividend Yield %            1 non-null      float64
 8   ROCE %                      1 non-null      float64
 9   ROE %                       1 non-null      float64
 10  Face Value in Rupees        1 non-null      float64
dtypes: float64(5), int64(5), object(1)
memory usage: 220.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 1

'../Output/RawReports-03/TCS-consolidated-20240903-210830.xlsx'